In [1]:
!nvidia-smi

Mon Nov 15 11:52:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%capture
!git clone -b master https://github.com/charles9n/bert-sklearn
!cd bert-sklearn; pip install .

In [13]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.metrics import classification_report
from sklearn import metrics

In [4]:
device = torch.device("cuda")
torch.cuda.is_available()

True

### Loading Dataset and Preprocessing

In [5]:
train = pd.read_csv("train.tsv", sep='\t', names=["text", "label"])
dev = pd.read_csv("dev.tsv", sep='\t', names=["text", "label"])
test = pd.read_csv("test.tsv", sep='\t', names=["text", "label"])

In [6]:
def change_label(df):
  df.loc[df['label'] >= 0.8, 'label'] = 1
  df.loc[df['label'] < 0.8, 'label'] = 0
  df['label'] = df['label'].astype(int)

In [7]:
for df in [train, dev, test]:
  change_label(df)

### Merging Train and dev set

In [8]:
train = train.append(dev)

In [9]:
from bert_sklearn import BertClassifier

### Training and Prediction

In [10]:
model = BertClassifier(num_mlp_layers=1, max_seq_length=64)
# finetune model
model.fit(train['text'], train['label'])

# make predictions
y_pred = model.predict(test['text'])

Building sklearn text classifier...


100%|██████████| 231508/231508 [00:00<00:00, 2121616.18B/s]


Loading bert-base-uncased model...


100%|██████████| 433/433 [00:00<00:00, 83243.97B/s]


Using mlp with D=768,H=500,K=2,n=1
Loading Pytorch checkpoint

train data size: 19125, validation data size: 2125



...


Epoch 2, Train loss: 0.2742, Val loss: 0.4494, Val accy: 82.02%



Training  :   0%|          | 0/597 [00:00<?, ?it/s]

Validating:   0%|          | 0/266 [00:00<?, ?it/s]


Epoch 3, Train loss: 0.1577, Val loss: 0.5596, Val accy: 81.18%



Predicting:   0%|          | 0/482 [00:00<?, ?it/s]

### Preformance check and accuracy

In [14]:
# score model on test data
model.score(test['text'], test['label'])



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Testing:   0%|          | 0/482 [00:00<?, ?it/s]


Loss: 0.5636, Accuracy: 81.17%


81.16883116883116

In [17]:
print("Accuracy: %0.2f%%"%(metrics.accuracy_score(y_pred, test['label']) * 100))

target_names = ['not well formed', 'well formed']
print(classification_report(test['label'], y_pred, target_names=target_names))

Accuracy: 81.17%
                 precision    recall  f1-score   support

not well formed       0.91      0.77      0.83      2370
    well formed       0.71      0.87      0.78      1480

       accuracy                           0.81      3850
      macro avg       0.81      0.82      0.81      3850
   weighted avg       0.83      0.81      0.81      3850

